<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex13notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2022 ex13notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo13.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2022

In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の DCT と FFT の関数
from scipy.fft import dct, fft, ifft

---
## 直交展開，離散コサイン変換，離散フーリエ変換
---



---
### ベクトルの直交展開 ― 成分を分析したい ー

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/soup.png" align="right">

スープの味が，塩味や甘味などのいくつかの味の成分の量で決まっており，それらの量は，それぞれの味に対応した調味料の含有量で測れるとしましょう．このとき，あるスープに各成分がどれくらいずつ含まれているかがわかれば，図のようなグラフを描けるでしょう．

このような情報が得られることには，次のような利点があります：

- そのスープの特徴（どんな成分を多く含んでいるのか）がわかる
- ある成分の量を増減させることで味を調節できる
- 成分の量がわかっているので，別のところでスープの味を再現できる

というわけで，ベクトルをいくつかの「成分」に分解して表す方法を考えてみましょう．


$D$ 次元ベクトル $\mathbf{x} = (x_1, x_2, \ldots, x_D)$ を，いくつかの成分 $\mathbf{u}_1, \mathbf{u}_2, \ldots$ を用いて，$\mathbf{x} = c_1\mathbf{u}_1 + c_2\mathbf{u}_2 + \cdots$ のように表すことを考えます．ただし，$\mathbf{x}$ や $\mathbf{u}_1, \mathbf{u}_2,\ldots$ は $D$ 次元実ベクトルであり，したがって，係数 $c_1, c_2, \ldots$ も実数とします．

このとき，線形代数で学んだように，次のことが言えます：

> $D$ 次元ベクトル $\mathbf{u}_1, \mathbf{u}_2, \ldots, \mathbf{u}_D$ が **線形独立** （**一次独立**）であれば，任意の $D$ 次元ベクトル $\mathbf{x}$ をこれらの **線形結合** （**一次結合**）として
> 
> $$
\mathbf{x} = c_1\mathbf{u}_1 + c_2\mathbf{u}_2 + \cdots + c_D\mathbf{u}_D \qquad (1)
$$
>
> と一意に表せる．このようなベクトルの集合 $\{ \mathbf{u}_1, \mathbf{u}_2, \ldots, \mathbf{u}_D \}$ を **基底** という．

これは，言い換えれば，$D$ 個の値から成る任意のベクトルを，$\mathbf{u}_1, \ldots, \mathbf{u}_D$ という成分の和で表せるということです．式 $(1)$ のように表すことを，「ベクトル $\mathbf{x}$ を基底 $\{\mathbf{u}_1, \ldots, \mathbf{u}_D\}$ で**展開**する」といいます．


<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/base-3d.png">

上の左と真ん中は，3次元空間中のあるベクトル $\mathbf{x}$ とその $\{\mathbf{u}_1, \mathbf{u}_2, \mathbf{u}_3\}$ による展開を図で表現したものです．
二つの図のベクトル $\mathbf{x}$ は同じものですが，左と真ん中では基底が異なっています．

一方，右の図は，$\mathbf{u}_3$ が $\mathbf{u}_1$ と $\mathbf{u}_2$ の線形結合で表せる（これらが線形独立でない）場合を示しています．この場合，$c_1\mathbf{u}_1 + c_2\mathbf{u}_2 + c_3\mathbf{u}_3$ の $c_1, c_2, c_3$ をどのように変えても図のピンクの平面上から外れることはできず，$\mathbf{x} = c_1\mathbf{u}_1 + c_2\mathbf{u}_2 + c_3\mathbf{u}_3$ となる $c_1, c_2, c_3$ が存在しません．

基底は，互いに直交しかつ大きさが $1$ のベクトルで構成するのが好都合です．つまり，基底 $\{ \mathbf{u}_1, \ldots, \mathbf{u}_D \}$ を構成するすべてのベクトルが

$$
\quad \mathbf{u}_i\cdot\mathbf{u}_j = 
    \begin{cases}
    0 & (i \ne j)\\
    1 & (i = j)
    \end{cases} \quad (2)
$$

を満たすようにする，ということです（任意のベクトル $\mathbf{u}$ に対して $\mathbf{u}\cdot\mathbf{u} = \Vert\mathbf{u}\Vert^2$ が成り立つので，$\mathbf{u}\cdot\mathbf{u} = 1 \Leftrightarrow \Vert\mathbf{u}\Vert = 1$ となります）．

その理由としては，まず第1に，基底を構成するベクトルが互いに直交していないと，一つの展開係数の値が変わると他の係数の値も変わってしまう，ということがあげられます．ある「成分」の量を増減したら他の「成分」の量も変化する，というのでは「成分」として扱いづらいですね．

第2の理由は，基底ベクトルだちが互いに直交してかつ大きさも $1$ だと，

$$
\begin{aligned}
\mathbf{u}_d\cdot\mathbf{x} &= \mathbf{u}_d\cdot (c_1\mathbf{u}_1+c_2\mathbf{u}_2+ \cdots + c_D\mathbf{u}_D) \\
&= \mathbf{u}_d\cdot(c_1\mathbf{u}_1) + \cdots + \mathbf{u}_d\cdot(c_d\mathbf{u}_d) + \cdots + \mathbf{u}_d\cdot(c_D\mathbf{u}_D) \\
&= c_d\mathbf{u}_d \cdot \mathbf{u}_d = c_d\Vert\mathbf{u}_d\Vert^2 = c_d 
\end{aligned}
$$

となるので，与えられたベクトル $\mathbf{x}$ と基底を構成する個々のベクトル $\mathbf{u}_d$ との内積 $\mathbf{x}\cdot\mathbf{u}_d$ を計算するだけで，展開の係数 $c_d$ （$d = 1, 2, \ldots, D$）が求まる，というものです．

以上のことから，ベクトルを展開したい場合，互いに直交しかつ大きさが $1$ のベクトルを集めた基底をよく用います．
互いに直交するベクトルから成る基底を **直交基底** といい，大きさが $1$ のベクトルから成る直交基底を **正規直交基底** といいます．直交基底を用いたベクトルの展開を **直交展開** といいます．

**問題1**

(1)

$$
\mathbf{u}_1 = \left( \frac{1}{\sqrt{2}}, \frac{1}{\sqrt{2}}, 0  \right),\quad
\mathbf{u}_2 = \left( \frac{1}{\sqrt{3}}, -\frac{1}{\sqrt{3}}, \frac{1}{\sqrt{3}}  \right),\quad
\mathbf{u}_3 = \left( \frac{1}{\sqrt{6}}, -\frac{1}{\sqrt{6}}, -\frac{2}{\sqrt{6}}  \right)
$$

とする．$\{\mathbf{u}_1, \mathbf{u}_2, \mathbf{u}_3\}$ が正規直交基底であることを示しなさい．

(2) (1)の結果を利用して，$\mathbf{x} = (2, 4, -1)$ を $\{\mathbf{u}_1, \mathbf{u}_2, \mathbf{u}_3\}$ で展開しなさい．つまり，$\mathbf{x} = c_1\mathbf{u}_1 + c_2 \mathbf{u}_2 + c_3 \mathbf{u}_3$ を満たす $c_1, c_2, c_3$ を求めなさい．

(3) (2) で求めた $c_1\mathbf{u}_1 + c_2\mathbf{u}_2 + c_3\mathbf{u}_3$ が $\mathbf{x}$ に等しいことを確かめなさい．

(4) $\mathbf{x} = \left( 2\sqrt{3}, 0, -2\sqrt{3} \right)$ に対して，(2), (3) と同じことをしなさい．

4次元ベクトルの例も挙げます．計算をさぼるために，NumPy 使いましょう．

In [ ]:
u1 = np.array([ 1/2,  1/2,  1/2,  1/2])
u2 = np.array([ 1/2,  1/2, -1/2, -1/2])
u3 = np.array([ 1/2, -1/2,  1/2, -1/2])
u4 = np.array([ 1/2, -1/2, -1/2,  1/2])

これらは正規直交基底を成しています．内積を計算して確認してみましょう．

In [ ]:
print(u1 @ u1)
print(u2 @ u1, u2 @ u2)
print(u3 @ u1, u3 @ u2, u3 @ u3)
print(u4 @ u1, u4 @ u2, u4 @ u3, u4 @ u4)

この基底を使って次のベクトルを展開してみましょう．

In [ ]:
x = np.array([5, 9, 6, 3])

展開係数を計算すると次のようになります．

In [ ]:
c1 = u1 @ x
c2 = u2 @ x
c3 = u3 @ x
c4 = u4 @ x
print(c1, c2, c3, c4)

正しい展開かどうか，検算してみましょう．

In [ ]:
xx = c1 * u1 + c2 * u2 + c3 * u3 + c4 * u4
print(xx)

正規直交基底を構成するベクトルを並べた行列を作ると便利です．以下，ベクトルを列ベクトルで表すことにします．$\{\mathbf{u}_1, \ldots, \mathbf{u}_D\}$ が正規直交基底であるとき，

$$
U = \begin{pmatrix}
\mathbf{u}_1 & \mathbf{u}_2 & \cdots & \mathbf{u}_D
\end{pmatrix}
$$

という $D\times D$ 行列を考えると，展開係数は次式で求まります．

$$
\begin{pmatrix}
c_1 \\ c_2 \\ \vdots \\ c_D
\end{pmatrix} =
U^{\top}\mathbf{x} \qquad (3)
$$

ちなみに，定義から $U$ について次のことが成り立ちます．

$$
U^{\top}U = \begin{pmatrix}
\mathbf{u}_1^{\top} \\ \mathbf{u}_2^{\top} \\ \vdots \\ \mathbf{u}_D^{\top}
\end{pmatrix}
\begin{pmatrix}
\mathbf{u}_1 & \mathbf{u}_2 & \cdots & \mathbf{u}_D
\end{pmatrix} =
\begin{pmatrix}
\mathbf{u}_1^{\top}\mathbf{u}_1 & \mathbf{u}_1^{\top}\mathbf{u}_2 & \cdots & \mathbf{u}_1^{\top}\mathbf{u}_D \\
\mathbf{u}_2^{\top}\mathbf{u}_1 & \mathbf{u}_2^{\top}\mathbf{u}_2 & \cdots & \mathbf{u}_2^{\top}\mathbf{u}_D \\
\vdots & & \ddots & \vdots\\
\mathbf{u}_D^{\top}\mathbf{u}_1 & \mathbf{u}_D^{\top}\mathbf{u}_2 & \cdots & \mathbf{u}_D^{\top}\mathbf{u}_D \\
\end{pmatrix} = I \qquad (4)
$$

このような性質をもつ行列のことを，（正規）**直交行列** といいます．詳しくは線形代数の書籍等を参照してね．

行列 `U` を作って上と同じことをやってみると，こんな感じ．

In [ ]:
U = np.vstack([u1, u2, u3, u4]).T
print(U)

In [ ]:
U.T @ U

In [ ]:
print(U.T @ x)

---
### 離散コサイン変換 (DCT, Discrete Cosine Transform)

上記の例では適当に定めた正規直交基底を用いてベクトルを展開していましたが，実際には，目的に応じて様々な基底が用いられます（よ）．ここでは，展開の方法の一例として，**離散コサイン変換**（DCT, Discrete Cosine Transform）を紹介します．離散コサイン変換は，音声や画像などのデータの情報処理においてよく用いられるものです．

※よだんだよん: 上記の4次元の例で出てきた基底は，「アダマール変換」と呼ばれるもので用いられる正規直交基底になっています（Wikipedia の「[アダマール変換](https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%80%E3%83%9E%E3%83%BC%E3%83%AB%E5%A4%89%E6%8F%9B)」）．
ちなみに，主成分分析では分散共分散行列の固有ベクトルを求めていましたが，実はあれらは与えられたデータの空間における正規直交基底を成しています．主成分分析は，データから正規直交基底を構成する方法ともみなせます．



> **離散コサイン変換**
>
> $N$ 次元ベクトル $\mathbf{u}_0, \mathbf{u}_1, \ldots, \mathbf{u}_{N-1}$ を次のように定めると，$\{\mathbf{u}_0, \mathbf{u}_1, \ldots, \mathbf{u}_{N-1}\}$ は正規直交基底となる（基底の番号が $0$ から $N-1$ までであることに注意）．
>
> $$
\begin{aligned}
\mathbf{u}_0 &= \sqrt{\frac{1}{N}}(1, 1, \ldots, 1) & (5)\\
\mathbf{u}_k &= \sqrt{\frac{2}{N}}\left( \cos{\frac{\pi k}{2N}}, \cos{\frac{3\pi k}{2N}}, \ldots, \cos{\frac{(2N-1)\pi k}{2N}} \right) \quad (k = 1, 2, \ldots, N-1) & (6)
\end{aligned}
$$
>
> したがって，$\{ \mathbf{u}_k \}$ を用いると，任意の $N$ 次元実ベクトル $\mathbf{x} = (x_0, x_1, \ldots, x_{N-1})$ （ $\mathbf{x}$ の要素の添字も $0$ から $N-1$ まで）を
>
> $$
\mathbf{x} = c_0\mathbf{u}_0 + c_1\mathbf{u}_1 + \cdots + c_{N-1}\mathbf{u}_{N-1} \qquad (7)
$$
>
> と一意に展開できる．ただし，展開係数 $c_k = \mathbf{u}_k\cdot\mathbf{x}$ である．$\mathbf{x}$ から $\mathbf{c} = (c_0, c_1, \ldots, c_{N-1})$ を求める演算を離散コサイン変換という（注）．

<span style="font-size: 75%">
※注: 実は，離散コサイン変換の定義には，式の形が少し異なるバリエーションがいくつかあります．ここで説明しているのは，DCT-II や Type II と呼ばれるタイプのもの（を正規直交基底となるようにしたもの）です．
</span>

この式だけではどんなものか分かりづらいので，値を求めたりグラフに描いたりしてみましょう．ここでは，[SciPy](https://scipy.org/) の関数である [`scipy.fft.dct`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.dct.html) を使うことにします．

$N=16$ のときの基底の値を表示させてみるとこんなん：

In [ ]:
N = 16
U_dct = dct(np.eye(N), norm='ortho')
U_dct

`U_dct` の列に基底ベクトルが格納されています．`U_dct[:, 0]` が $\mathbf{u}_0 = \left(\frac{1}{4}\ \frac{1}{4}\ \cdots\ \frac{1}{4}\right)^{\top}$ ．

本当に正規直交基底になっているか確認しましょう．

In [ ]:
U_dct.T @ U_dct

ちゃんと単位行列になっていて（対角要素が $1$ で，それ以外の要素には計算誤差が含まれますが $0$ になっています），式$(4)$が成り立っていることが分かりますね．

式$(5),(6)$の各基底ベクトルをグラフに描いてみると，次のようになります．

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(16, 12))
xp = np.linspace(0, N-1, num=200)
for i in range(4):
    for j in range(4):
        k = i*4 + j
        ax[i][j].plot(U_dct[:, k], 'o', color='blue', label=f'u{k}')
        if k == 0:
            f = np.sqrt(1/N) * np.ones_like(xp)
        else:
            f = np.sqrt(2/N) * np.cos(k*np.pi*(2*xp+1)/(2*N))
        ax[i][j].plot(xp, f, color='gray')
        ax[i][j].set_ylim(-0.5, 0.5)
        ax[i][j].axhline(0, color='gray')
        ax[i][j].legend()
plt.show()

基底ベクトルの要素の値は青い点として描かれています．これらの点は，灰色で描かれた正弦波（この場合は $\cos$ 関数）を間隔 $1$ でサンプリングしたものになっています．$\mathbf{u}_1$ は周期 $2N$ の波，$\mathbf{u}_2$ は周期 $2N/2$ の波，$\mathbf{u}_3$ は周期 $2N/3$，...，というように，基底ベクトルの番号が大きくなるほど，対応する波の周期が短く（周波数が高く）なっています．

このような基底を用いて展開するということは，下図に示すように，与えられたベクトルを様々な周期の波の成分の和として表すことに相当します．展開係数の値を調べることで，どんな周期の波の成分をどのくらい含んでいるのかを知ることができます．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/DCTexpansion.png">

実際の例を見てみましょう．

$$
\mathbf{x} = (0, -2, 0, -4, 10, 12, 10, 12, 8, 5, 4, 3, -2, 0, 2, 0)
$$

という $N = 16$ 次元ベクトルを離散コサイン変換してみます．

In [ ]:
x16 = np.array([0, -2, 0, -4, 10, 12, 10, 12, 8, 5, 4, 3, -2, 0, 2, 0])
print(x16)

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(x16, '-o', color='red', linewidth=2)
ax.axhline(0, color='gray')
ax.set_ylim(-5, 15)
plt.show()

[`scipy.fft.dct`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.dct.html) に np.array を渡すと展開係数が返ってきます．

In [ ]:
# x16 の離散コサイン変換
c16 = dct(x16, norm='ortho')

# 上記は次の計算と等価だがより計算量の少ないアルゴリズムを用いているので，
# N が大きい場合は上記の方が圧倒的に速い
# c16 = U_dct.T @ x16

print(c16)

# 展開係数の可視化
fig, ax = plt.subplots(figsize=(8, 6))
ax.stem(c16, markerfmt='o', use_line_collection=True)
ax.set_ylim(-20, 20)
plt.show()

定数成分である $\mathbf{u}_0$ を除くと，$\mathbf{u}_2$ の係数の絶対値が最も大きくなっています．元の波形からも想像できるように，$\mathbf{u}_2$ の波（を正負逆にしたもの）の成分が最も多く含まれているといえます．

次に，展開の様子を視覚的に理解するために，

$$
\mathbf{x} = c_0\mathbf{u}_0 + c_1\mathbf{u}_1 + \cdots + c_{N-1}\mathbf{u}_{N-1}
$$

の右辺を途中までで打ち切ったものを計算してグラフに描いてみましょう．



In [ ]:
fig, ax = plt.subplots(5, 2, figsize=(16, 12))

### 左側 u3 まで

# z = c[0]*u[0] + c[1]*u[1] + c[2]*u[2] + c[3]*u[3]
z = U_dct[:, :4] @ c16[:4]

ax[0, 0].plot(x16, '-o', color='red', label='x16')
ax[0, 0].plot(z, '-o', color='blue', label='z')
ax[0, 0].axhline(0, color='gray')
ax[0, 0].set_ylim(-5, 15)
ax[0, 0].legend()

for k in range(4):
    ax[k+1, 0].plot(c16[k]*U_dct[:, k], '-o', color='blue', label=f'c{k} * u{k}')
    ax[k+1, 0].axhline(0, color='gray')
    ax[k+1, 0].set_ylim(-5, 15)
    ax[k+1, 0].legend()

### 右側 u7 まで

# z = c[0]*u[0] + ... + c[7]*u[7]
z = U_dct[:, :8] @ c16[:8]

ax[0, 1].plot(x16, '-o', color='red', label='x16')
ax[0, 1].plot(z, '-o', color='blue', label='z')
ax[0, 1].axhline(0, color='gray')
ax[0, 1].set_ylim(-5, 15)
ax[0, 1].legend()

for k in range(4, 8):
    ax[k-3, 1].plot(c16[k]*U_dct[:, k], '-o', color='blue', label=f'c{k} * u{k}')
    ax[k-3, 1].axhline(0, color='gray')
    ax[k-3, 1].set_ylim(-5, 15)
    ax[k-3, 1].legend()

plt.show()

上図の左は，$\mathbf{z} = c_0\mathbf{u}_0 + c_1\mathbf{u}_1 + \cdots + c_{3}\mathbf{u}_{3}$，右は，$\mathbf{z} = c_0\mathbf{u}_0 + c_1\mathbf{u}_1 + \cdots + c_{7}\mathbf{u}_{7}$ です．グラフの2行目以下には $c_k\mathbf{u}_k$ も描いてあります．

左上の `z` は，左下の4つの波を足し合わせたものとなっています．番号が小さい基底ベクトル（＝周期の長い成分）のみを足し合わせているため，元の `x16` の波形のうち，変化が激しい部分（横軸の値の短い範囲で値が大きく変化する部分）をうまくあらわせていません．

右上の `z` は，左上の `z` にさらに右下の4つの波も足し合わせたものです．より周期が短い成分も加わることで，より元の波形に近くなっていることが分かります．

さらに $\mathbf{u}_8$ から $\mathbf{u}_{15}$ までの成分も加えると，下図のように元の波形を完全に再現することができます．

In [ ]:
fig, ax = plt.subplots(5, 2, figsize=(16, 12))

### 左側 u11 まで

# z = c[0]*u[0] +  ... + c[11]*u[11]
z = U_dct[:, :12] @ c16[:12]

ax[0, 0].plot(x16, '-o', color='red', label='x16')
ax[0, 0].plot(z, '-o', color='blue', label='z')
ax[0, 0].axhline(0, color='gray')
ax[0, 0].set_ylim(-5, 15)
ax[0, 0].legend()

for k in range(8, 12):
    ax[k-7, 0].plot(c16[k]*U_dct[:, k], '-o', color='blue', label=f'c{k} * u{k}')
    ax[k-7, 0].axhline(0, color='gray')
    ax[k-7, 0].set_ylim(-5, 15)
    ax[k-7, 0].legend()

### 右側 全部

# z = c[0]*u[0] + ... + c[15]*u[15]
z = U_dct @ c16

ax[0, 1].plot(x16, '-o', color='red', label='x16')
ax[0, 1].plot(z, '-o', color='blue', label='z')
ax[0, 1].axhline(0, color='gray')
ax[0, 1].set_ylim(-5, 15)
ax[0, 1].legend()

for k in range(12, 16):
    ax[k-11, 1].plot(c16[k]*U_dct[:, k], '-o', color='blue', label=f'c{k} * u{k}')
    ax[k-11, 1].axhline(0, color='gray')
    ax[k-11, 1].set_ylim(-5, 15)
    ax[k-11, 1].legend()

plt.show()

---
### 直交展開を利用したベクトルの近似

$N$ 次元のベクトル $\mathbf{x}$ は，$N$ 個の $N$ 次元ベクトルから成る正規直交基底 $\{\mathbf{u}_1, \mathbf{u}_2, \ldots, \mathbf{u}_{N}\}$ を用いて，次のように直交展開できます．


$$
\mathbf{x} = c_1\mathbf{u}_1 + c_2\mathbf{u}_2 + \cdots + c_N\mathbf{u}_N
$$

ただし，展開係数 $c_k = \mathbf{u}_k\cdot\mathbf{x}$ です（ $k=1,2,\ldots,N$ ）．

この式の右辺は，与えられた $N$ 個の基底ベクトル全ての線形結合になっており，左辺の $\mathbf{x}$ に等しくなります．前のセクションの最後の所で見たように，全ての基底ベクトルを用いるのではなく一部だけを用いる（上の式で用いない展開係数の値を $0$ とおく）と，それらの成分だけで $\mathbf{x}$ を近似することになります．

前のセクションでは番号の小さい方の成分を使う（番号の大きい方の成分を無視する）ことにしていましたが，一般には，どの番号の成分を使うかは任意です．$N$ 個のうち $H$ 個を使うとして，使う成分の番号を $I_1, I_2, \ldots, I_H$ とおくと，これらの成分による $\mathbf{x}$ の近似は次のように表せます．

$$
\mathbf{x} \approx c_{I_1}\mathbf{u}_{I_1} + c_{I_2}\mathbf{u}_{I_2} + \cdots + c_{I_H}\mathbf{u}_{I_H} \qquad (8)
$$

このとき，右辺が左辺の良い近似になる（注）ならば，この式を利用して，データの表現に必要な数値の数を減らすことができます．例えば，ほげお君が通信回線越しにあなたへ，$N = 10000$ 次元の画像データ（画素数 $100\times 100$ のグレイスケール画像の画素値）を何枚も送信しようとしているとしましょう．
ほげお君が元の画像データ $\mathbf{x}$ を送信する場合，1枚の画像あたり $10000$ 個の数値が必要です．しかし，これらの画像データを直交展開して $H=100$ 個の成分で元の画像が十分近似できるなら，ほげお君が $100$ 個の展開係数 $c_{I_1}, c_{I_2},\ldots,c_{I_{100}}$ の値を求めてこれらをあなたへ送信し，あなたがそれらを使って式$(8)$の右辺を計算する，というようにすることで，送信する数値の数を $H/N = 1/100$ に減らすことができます（ただし，ほげお君はあなたへあらかじめ $H$ 個の基底ベクトルを伝えておく必要があります）．

このように，直交展開は，データを成分に分けてデータの性質を分析するためだけでなく，画像や音声のようなデータの「データ圧縮」にも利用することができます．notebookB で画像圧縮への応用の例を紹介する予定です．

※ 注意: この説明は，「良い」の定義をちゃんとしてないいい加減なものになっています．ごく簡単に説明しておくと，実際には，$\mathbf{x}$ と右辺の値との間の誤差を $\Vert\mathbf{x} - (c_{I_1}\mathbf{u}_{I_1} + c_{I_2}\mathbf{u}_{I_2} + \cdots + c_{I_H}\mathbf{u}_{I_H})\Vert$ として，これを規準として考えます．


---
### 離散フーリエ変換 (DFT, Discrete Fourier Transform)


画像や音声などのデータの分析や処理のための道具の一つに **離散フーリエ変換** (DFT, Discrete Fourier Transform（よ）) というものがあります．離散コサイン変換同様，これも直交展開の一種です．ただし，複素数を使います．

※ よだんだよん: フーリエ変換やフーリエ級数展開の Fourier は，フランスの数学者 Joseph Fourier (1768 - 1830) に由来します．Wikipedia の「[ジョゼフ・フーリエ](https://ja.wikipedia.org/wiki/%E3%82%B8%E3%83%A7%E3%82%BC%E3%83%95%E3%83%BB%E3%83%95%E3%83%BC%E3%83%AA%E3%82%A8)」．

> **離散フーリエ変換**
>
> $N$ 次元ベクトル $\mathbf{u}_0, \mathbf{u}_1, \ldots, \mathbf{u}_{N-1}$ を次のように定めると，$\{\mathbf{u}_0, \mathbf{u}_1, \ldots, \mathbf{u}_{N-1}\}$ は直交基底となる．
>
> $$
\mathbf{u}_k = \left( e^{0\times i\omega_0k}, e^{1\times i\omega_0k}, e^{2\times i\omega_0k}, \ldots, e^{(N-1)\times i\omega_0k} \right) \quad (k = 0, 1, 2, \ldots, N-1) \qquad (9)
$$
>
> ただし，$i = \sqrt{-1}, w_0 = \frac{2\pi}{N}$ である．
> $\{ \mathbf{u}_k \}$ を用いると，任意の $N$ 次元ベクトル $\mathbf{x} = (x_0, x_1, \ldots, x_{N-1})$ を
>
> $$
\mathbf{x} = c_0\mathbf{u}_0 + c_1\mathbf{u}_1 + \cdots + c_{N-1}\mathbf{u}_{N-1} \qquad (10)
$$
>
> と一意に展開できる．展開係数 $c_k$ の値は次式で求められる．
>
> $$
c_k = \frac{1}{N}\mathbf{x}\cdot\mathbf{u}_k = \frac{1}{N}\sum_{n=0}^{N-1} x_{n}e^{-i\omega_0kn} \qquad (11)
$$
>
> $\mathbf{x}$ から $\mathbf{c} = (c_0, c_1, \ldots, c_{N-1})$ を求める演算を **離散フーリエ変換** という．また，展開係数のことを **フーリエ係数** という．

いろいろわけわかめなところがあると思いますので，説明していきます．

まず，この基底を構成するベクトルがどんなものなのかを理解するために，値を眺めたりグラフに描いたりしてみましょう．ここでは，[SciPy](https://scipy.org/) の関数である [`scipy.fft.fft`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.fft.html) を使っています．

$N = 4$ のときは， $\omega_0 = \frac{2\pi}{4} = \frac{\pi}{2}$ で，

$$
\begin{aligned}
\mathbf{u}_0 &= \left( e^{0\times\frac{0\pi}{2}i}, e^{1\times\frac{0\pi}{2}i}, e^{2\times\frac{0\pi}{2}i}, e^{3\times\frac{0\pi}{2}i} \right) = (1, 1, 1, 1)\\
\mathbf{u}_1 &= \left( e^{0\times\frac{\pi}{2}i}, e^{1\times\frac{\pi}{2}i}, e^{2\times\frac{\pi}{2}i}, e^{3\times\frac{\pi}{2}i} \right) = (1, i, -1, -i)\\
\mathbf{u}_2 &= \left( e^{0\times\frac{2\pi}{2}i}, e^{1\times\frac{2\pi}{2}i}, e^{2\times\frac{2\pi}{2}i}, e^{3\times\frac{2\pi}{2}i} \right) = (1, -1, 1, -1)\\
\mathbf{u}_3 &= \left( e^{0\times\frac{3\pi}{2}i}, e^{1\times\frac{3\pi}{2}i}, e^{2\times\frac{3\pi}{2}i}, e^{3\times\frac{3\pi}{2}i} \right) = (1, -i, -1, i)\\
\end{aligned}
$$

となります．

In [ ]:
# 4次元の DFT の基底
U_dft4 = np.conj(fft(np.eye(4)))
U_dft4

NumPy では，虚数単位を `j` で表します．上記の最初の列が $\mathbf{u}_0$ に対応しています．

次に，$N=16$ の基底の一部をグラフに描いてみましょう．

In [ ]:
# 16次元の DFT の基底のうち最初の4つ
N = 16
U_dft = np.conj(fft(np.eye(N)))
fig, ax = plt.subplots(2, 4, figsize=(16, 6))
xp = np.linspace(0, N-1, num=200)
for k in range(4):
    # 実部
    re = U_dft[:, k].real # 実部
    ax[0][k].plot(re, 'o', color='blue', label=f'Re u{k}')
    omega0 = 2*np.pi/N
    yp = np.cos(omega0*k*xp)
    ax[0][k].plot(xp, yp, color='gray')
    # 虚部
    im = U_dft[:, k].imag # 虚部
    ax[1][k].plot(im, 'o', color='red', label=f'Im u{k}')
    yp = np.sin(omega0*k*xp)
    ax[1][k].plot(xp, yp, color='gray')
    for i in range(2):
        ax[i][k].set_ylim(-1.4, 1.4)
        ax[i][k].axhline(0, color='gray')
        ax[i][k].legend()
plt.show()

<img width=500 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Euler.png" align="right">

上段が実部，下段が虚部で，左から順に $\mathbf{u}_0$, $\mathbf{u}_1$, $\mathbf{u}_2$, $\mathbf{u}_3$ を表しています．実部は $\cos$，虚部は $\sin$ の形をしています．このことは，オイラーの等式を用いて基底の式を変形してみると分かります（右図も参照）．

$\mathbf{u}_1$ の各要素は，複素平面上の単位円周上を $1$ からスタートして周期 $N$ で回転する点の座標を間隔$1$でサンプリングしたものとなっており，$\mathbf{u}_2$ の各要素は同様に周期 $N/2$ で（つまり2倍の速さで）回転する点の座標，$\mathbf{u}_3$ の各要素は周期 $N/3$ で回転する点の座標，...，というようになっています（よ）．つまり，番号の大きい基底ほど，対応する波の周期が短く（周波数が高く）なっています．

ただし，上記のことがいえるのは基底の番号 $k$ が $1$ 以上 $\frac{N}{2}$ 以下の場合です．$\frac{N}{2} < k \leq N-1$ のときは，$\mathbf{u}_k$ の各要素は周期 $N/(N-k)$ で単位円周上を逆向きに回転します（下図参照）．

離散コサイン変換の場合と同様に，これらの基底を用いた直交展開の展開係数（フーリエ係数）の値を調べることで，元のベクトルの中に様々な波の成分がどれくらい含まれているのかを知ることができます．


※よだんだよん: 複素平面上に $1$ を頂点に持つ正$N$角形を考えると，$\mathbf{u}_1$ の要素の値は，$1$ からスタートして反時計周りに一つずつ頂点を移動していったときの頂点の座標となっています．$\mathbf{u}_2$ は1つ飛ばし，$\mathbf{u}_3$ は2つ飛ばし...．


In [ ]:
# 16次元の DFT の基底のうち最後の4つ
N = 16
U_dft = np.conj(fft(np.eye(N)))
fig, ax = plt.subplots(2, 4, figsize=(16, 6))
xp = np.linspace(0, N-1, num=200)
for k in range(4):
    k2 = k + 12
    # 実部
    re = U_dft[:, k2].real # 実部
    ax[0][k].plot(re, 'o', color='blue', label=f'Re u{k2}')
    omega0 = 2*np.pi/N
    yp = np.cos(omega0*(k2-N)*xp)
    ax[0][k].plot(xp, yp, color='gray')
    # 虚部
    im = U_dft[:, k2].imag # 虚部
    ax[1][k].plot(im, 'o', color='red', label=f'Im u{k2}')
    yp = np.sin(omega0*(k2-N)*xp)
    ax[1][k].plot(xp, yp, color='gray')
    for i in range(2):
        ax[i][k].set_ylim(-1.4, 1.4)
        ax[i][k].axhline(0, color='gray')
        ax[i][k].legend()
plt.show()

ところで，注意深く式を眺めたひとはお気づきかもしれませんが，式$(11)$の $e$ の肩に乗っている式には負号がついています．これだと $\mathbf{u}_k\cdot\mathbf{x}$ と等しくなくておかしいように見えます．しかし，式$(11)$は正しい式です．実は，複素数を要素にもつベクトル $\mathbf{a} = (a_1, a_2, \ldots, a_D)$ と $\mathbf{b} = (b_1, b_2, \ldots, b_D)$ の内積は，次のように定義されます（注）．

$$
\mathbf{a}\cdot\mathbf{b} = \sum_{d=1}^D a_d \overline{{b}_d}
$$

$\overline{{b}_d}$ は，$b_d$ の複素共役です．そのため，式$(11)$ に $\overline{e^{i\omega_0kn}} = e^{-i\omega_0kn}$ が現れています．

※ 注意: この定義から，$\mathbf{a}\cdot\mathbf{b} = \overline{\mathbf{b}\cdot\mathbf{a}}$ が導かれます．

4次元のDFTの基底を例にして，これらが上記の内積の定義のもとで直交基底となっていることを確認してみます．

In [ ]:
# 4次元の DFT の基底
U_dft4

In [ ]:
U_dft4 @ np.conj(U_dft4)

上のセルの `np.conj` は，引数で渡された np.array の複素共役の値を返す NumPy の関数です． 対角要素以外が $0$ になっていることから，`U_dft4` の各列が互いに直交していることが分かります（注）．

※注意: 式$(9)$のベクトルたちは互いに直交してはいますが大きさは $1$ ではなく $N$ ですので，正規直交基底ではない直交基底です．全ての値を $\frac{1}{\sqrt{N}}$ すれば正規直交基底となりますが，離散フーリエ変換の定義はここで説明している形のものが一般的ですので，ここでもあえて正規直交基底にはしていません．

### 基底・係数の対称性と振幅・位相

前のセクションでも使っていた $N=16$ 次元のベクトルの例で，実際に離散フーリエ変換やってみましょう．

In [ ]:
print(x16)
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(x16, '-o', color='red', linewidth=2)
ax.axhline(0, color='gray')
ax.set_ylim(-5, 15)
plt.show()

[`scipy.fft.fft`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.fft.html) に np.array を渡すとフーリエ係数が返ってきます．

In [ ]:
# x16 の離散フーリエ変換
C16 = fft(x16)

# 上記は次の計算と等価だがより計算量の少ないアルゴリズムを用いているので，
# N が大きい場合は上記の方が圧倒的に速い
#C16 = np.conj(U_dft.T) @ x16

print(C16)

# フーリエ係数の可視化（実部と虚部）
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].stem(C16.real, markerfmt='o', use_line_collection=True, label='Re Ck')
ax[0].set_ylim(-60, 60)
ax[0].legend()
ax[1].stem(C16.imag, markerfmt='o', use_line_collection=True, label='Im Ck')
ax[1].set_ylim(-60, 60)
ax[1].legend()
plt.show()

このグラフを見ると，得られたフーリエ係数の値に対称性があることが分かります（実部のグラフは $c_0$ を除き $k = \frac{N}{2} = 8$ を軸として左右対称，虚部の方は符号を反転したうえで左右対称）．
前のセクションで，離散フーリエ変換の基底ベクトル $\mathbf{u}_k$ のうち，$\frac{N}{2} < k \leq N-1$ のものは，「逆向きに回転する」という説明をしました．このことを式で表すと，次のようになります（導出は省略します）．

$$
\mathbf{u}_{N-k} = \overline{\mathbf{u}_k} \qquad (k = 1, 2, \ldots, N-1)
$$

この性質からさらに，要素が実数のベクトル $\mathbf{x}$ の
フーリエ係数には

$$
c_{N - k} = \overline{c_{k}} \qquad (k = 1, 2, \ldots, N-1)
$$

という対称性があることが導かれます（これも導出は省略します）．上記のグラフの対称性は，これを反映したものです．





一方，ある複素数 $z$ の偏角が $\arg z = \phi$ であるとき，$z = |z|e^{i\phi}$ より $\overline{z} = |z|e^{-i\phi}$ と表せます．ゆえに，

$$
\begin{aligned}
ze^{i\theta} + \overline{z}e^{-i\theta} &= |z|e^{i(\theta+\phi)} + |z|e^{-(\theta+\phi)} = |z|(e^{\theta+\phi} + e^{-(\theta+\phi)} )\\
&= 2|z|\cos{(\theta+\phi)}
\end{aligned}
$$

と変形できます（最後の等号は，オイラーの等式より導かれる関係式 $\frac{e^{i\theta}+e^{-i\theta}}{2} = \cos\theta$ より）．

以上のことを用いると，$\mathbf{u}_k$ の要素と $c_k$ について，

$$
\begin{aligned}
c_ke^{i\omega_0kn} + c_{N-k}e^{i\omega_0(N-k)n} &= c_ke^{i\omega_0kn} + \overline{c_{k}}\overline{e^{i\omega_0kn}} = c_ke^{i\omega_0kn} + \overline{c_{k}}e^{-i\omega_0kn}\\
&= 2|c_k|\cos{(\omega_0kn+\arg c_k)} \qquad (k = 1, 2, \ldots, N-1, n = 1, 2, \ldots, N-1)
\end{aligned}
$$

と表せます．

したがって，$c_k\mathbf{u}_k + c_{N-k}\mathbf{u}_{N-k}$ は，「振幅 $2|c_k|$ ，位相 $\arg c_k$ で周期 $N/k$ の $\cos$」を表すことになります．つまり，離散フーリエ変換は，与えられたベクトル $\mathbf{x}$ を，

$$
\begin{array}{ll}
\mbox{定数:} & c_0\mathbf{u}_0\\
\mbox{周期 $N$ の波:} & c_1\mathbf{u}_1 + c_{N-1}\mathbf{u}_{N-1} \\
\mbox{周期 $N/2$ の波:} & c_2\mathbf{u}_2 + c_{N-2}\mathbf{u}_{N-2} \\
\mbox{周期 $N/3$ の波:} & c_3\mathbf{u}_3 + c_{N-3}\mathbf{u}_{N-3}\\
: & :
\end{array}
$$

の和として表しているといえます．

次のセルを実行すると，`x16` に格納された $N=16$ 次元ベクトル $\mathbf{x}$ から求めたフーリエ係数 $c_0, c_1, c_2, \ldots, c_{15}$ を用いて

$$
\mathbf{z} = c_0\mathbf{u}_0 + (c_1\mathbf{u}_1 + c_{N-1}\mathbf{u}_{N-1}) + (c_2\mathbf{u}_2 + c_{N-2}\mathbf{u}_{N-2}) + (c_3\mathbf{u}_3 + c_{N-3}\mathbf{u}_{N-3})
$$

という $16$ 次元ベクトルを求めた結果を図示します．

In [ ]:
# x16 を k = 0, 1, 2, 3, 13, 14, 15 の成分のみで近似
 
N = 16
U_dft = np.conj(fft(np.eye(N)))
C16 = fft(x16)

C16_truncated = np.copy(C16)
C16_truncated[3+1:N-3] = 0
z = ifft(C16_truncated).real
print(z)

fig = plt.figure(figsize=(8, 16))

ax = fig.add_subplot(5, 1, 1)
ax.plot(x16, '-o', color='red', label='x16')
ax.plot(z, '-o', color='blue', label='z')
ax.axhline(0, color='gray')
ax.set_ylim(-10, 15)
ax.legend()

ax = fig.add_subplot(5, 1, 2)
v = (C16[0] * U_dft[:, 0]).real / N
ax.plot(v, '-o', color='blue', label=f'c0*u0')
ax.axhline(0, color='gray')
ax.set_ylim(-10, 15)
ax.legend()

for k in range(1, 4):
    ax = fig.add_subplot(5, 1, k+2)
    # c_k * u_k + c_{N-k} * u_{N-k} の計算
    v = (C16[k] * U_dft[:, k] + C16[N-k] * U_dft[:, N-k]).real / N
    ax.plot(v, '-o', color='blue', label=f'c{k}*u{k} + c{N-k}*u{N-k}')
    ax.axhline(0, color='gray')
    ax.set_ylim(-10, 15)
    ax.legend()

plt.show()

一番上のグラフは，$\mathbf{x}$ と $\mathbf{z}$ を重ねて描いたものです．$\mathbf{z}$ は，$\mathbf{x}$ を「定数」+「周期 $16$ の波」 + 「周期 $16/2$ の波」 + 「周期 $16/3$ の波」のみで近似したものなので，より周期の短い成分が失われていることが分かります．

2つ目以降には，$c_0\mathbf{u}_0$, $c_1\mathbf{u}_1 + c_{N-1}\mathbf{u}_{N-1}$, $c_2\mathbf{u}_2 + c_{N-2}\mathbf{u}_{N-2}$, $c_3\mathbf{u}_3 + c_{N-3}\mathbf{u}_{N-3}$ の 4つが描いてあります．例えば $c_1\mathbf{u}_1 + c_{N-1}\mathbf{u}_{N-1}$ は，$\mathbf{u}$ の実部が表す波の振幅を $2|c_1|$ 倍して，位相を $\arg c_1$ だけずらしたものになっています（上下に引き伸ばされ，左右にずれている）．

### スペクトル

上述のように，離散フーリエ変換では，与えられたベクトルを

- 定数成分
- 周期 $N$，振幅 $|c_1|$，位相 $\arg c_1$ の波
- 周期 $N/2$，振幅 $|c_2|$，位相 $\arg c_2$ の波
- 周期 $N/3$，振幅 $|c_3|$，位相 $\arg c_3$ の波
- ...

に分解して（これらの和として）表します．したがって，離散フーリエ変換の結果を分析する際には，得られたフーリエ係数 $c_k$ の値そのものよりも，その絶対値 $|c_k|$ と偏角 $\arg c_k$ を見る方が便利なことがよくあります．
横軸に成分の番号 $k$ をとり，縦軸に $|c_k|$ の値をプロットしたグラフを **振幅スペクトル** といい， $\arg c_k$ をプロットしたグラフを **位相スペクトル** といいます（注）．


※ 注意: 厳密な言い方をするとグラフではなくフーリエ係数の絶対値の集合 $\{ |c_k| \}$ を振幅スペクトル，偏角の集合 $\{ \arg c_k \}$ を位相スペクトルといいますが，ここではわかりやすさを重視して，それらのグラフを〇〇スペクトルと呼ぶと説明しています．



上のベクトルの例で振幅スペクトル，位相スペクトルを描いてみると，次のようになります．

In [ ]:
amplitude = np.abs(C16) # 絶対値
phase = np.angle(C16) # 偏角
print(amplitude)
print(phase)

# フーリエ係数の可視化（振幅と位相）
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].stem(amplitude, markerfmt='o', use_line_collection=True, label='|Ck|')
ax[0].set_ylim(0, 60)
ax[0].legend()
ax[1].stem(phase, markerfmt='o', use_line_collection=True, label='arg Ck')
ax[1].set_ylim(-3.5, 3.5)
ax[1].legend()
plt.show()

演習の方では，実際の音のデータなどを離散フーリエ変換してスペクトルを描き，分析してみる予定です．